Setup for the KSQL API:

In [1]:
from ksql import KSQLAPI
client = KSQLAPI('http://localhost:8088')

In [ ]:
client.create_stream(table_name='creditcardfraud',
                     columns_type=['Id bigint', 'Time int', 'V1 double', 'V2 double', 'V3 double', 'V4 double', 'V5 double', 'V6 double', 'V7 double', 'V8 double', 'V9 double', 'V10 double', 'V11 double', 'V12 double', 'V13 double', 'V14 double', 'V15 double', 'V16 double', 'V17 double', 'V18 double', 'V19 double', 'V20 double', 'V21 double', 'V22 double', 'V23 double', 'V24 double', 'V25 double', 'V26 double', 'V27 double', 'V28 double', 'Amount double', 'Class string'],
                     topic='creditcardfraud',
                     value_format='DELIMITED')

In [ ]:
client.ksql('show streams')

Execute sql query and keep listening streaming data:

In [ ]:
TODO How to embed ' ???
client.ksql('SET 'auto.offset.reset'='earliest'');

In [2]:
query = client.query('select * from creditcardfraud LIMIT 3')
for item in query: print(item)

{"row":{"columns":[1544795891977,null,1,0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942
518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.6178008557
62348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960
255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.1891148
43888824,0.133558376740387,-0.0210530534538215,149.62,"0"]},"errorMessage":null,"finalMessage":null}
{"row":{"columns":[1544795892717,null,1,0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942
518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.6178008557
62348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992